In [636]:
import gmsh
import numpy as np


# https://gmsh.info/doc/texinfo/gmsh.html

In [637]:
# user inputs
filename = 'test.msh'

In [638]:
gmsh.initialize()
gmsh.open(filename)
entities = gmsh.model.getEntities() # vector of [dim, tag] pairs for each entity - points,surfaces,vols. tags are the ids of the entity - e.g. point 1, surface 1, volume 1.
                                    # each dimension has a new set of tags - e.g. you can have point 1 and surface 1 since surf and point are in different dimensions
print(entities)


Info    : Reading 'test.msh'...
Info    : 27 entities
Info    : 313 nodes
Info    : 1603 elements
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1)]
Info    : Done reading 'test.msh'


In [639]:
class face():
  # internal face - has a positive neighbor id
  # boundary condition face - has a negative neighbor id
  def __init__(self, nodes: list, owner: int, neighbor: int, face_id: int):
    self.face_id = face_id
    self.nodes = nodes
    self.own_id = owner # element eid of the owner
    self.neb_id = neighbor # element eid of the neighbor
    self.bndry = None # boundary that the face connects to
  def set_bndry(self, bnd_id: int):
    self.bnd_id = bnd_id

class element():
  def __init__(self, nodes: list, desc: str, ele_type: int, eid: int, sn: list):
    self.nodes = nodes # node tags for each node of this element
    self.sn = sn # surface node indexes for each node in this element.
    self.desc = desc # basic str describing thios element
    self.ele_type = ele_type # numeric type of this element
    self.eid = eid # mesh id of this elemeent
    self.num_surfs = len(sn) # number of surfaces on this element
    self.node_coords = np.array([])
    self.entity_dims = [] # dimension of the entity each node appears on/in
    self.entity_tags = [] # tag of the entity each node appears on/in
    self.entity_tag_pairs = [] # tuples pairing the entity it appears on/in and dim
    self.surfaceNodeTags = [] # node tags for each surface [[node1,node2,node3, node4], [], [], ...]
    self.assign_surface_node_tags()
    self.get_node_info()

    # data for boundaries
    self.is_boundary = [False]*len(sn) # true or false if the surfaces of this element connect to bndry
    self.boundary_ids = [None]*len(sn) # boundary ids (none if not bndry but sid if it is)

    # data for internal faces
    self.is_face = [False]*len(sn) # bool whether or not it is an internal face.
    self.neighbor_ids = [None]*len(sn) # ids of the neighboring element -- element eid

    # data for all faces
    self.face_ids = [None]*len(sn) # ids of the faces around this cell (including boundary and internal faces.)

  def verify_neighbors_and_faces(self):
    for idx, b in enumerate(self.is_boundary):
      if (b == True) & (self.is_face[idx] == True):
        raise Exception("eid =" + str(self.eid) + "A surface cannot be both a boundary and an internal face.")
      if (b == False) & (self.is_face[idx] == False):
        raise Exception("eid =" + str(self.eid) + "A surface must be either a boundary or a face.")

  def assign_neighbor_info(self, nodeTags: list, neighbor_id: int, face_id: int):
    for idx, this_node_tags in enumerate(self.surfaceNodeTags):
      sorted_this_node_tags = sorted(this_node_tags)
      sorted_input_node_tags = sorted(nodeTags)
      if sorted_this_node_tags == sorted_input_node_tags: # this is the correct surface.
        self.is_face[idx] = True
        self.face_ids[idx] = face_id
        self.neighbor_ids[idx] = neighbor_id

  def determine_if_boundary(self,surfaces):
    for idx, surfNodes in enumerate(self.surfaceNodeTags):
      # check if surfNodes is in any of surfaces.nodeTags
      for surf in surfaces:
        huge_set = set(surf.nodeTags)
        if all(item in huge_set for item in surfNodes):
          sid = surf.id
          self.boundary_ids[idx] = sid
          self.is_boundary[idx] = True

  def assign_surface_node_tags(self):
    # order = np.argsort(self.nodes)
    # self.nodes = sorted(self.nodes)
    # for surf_idx, surf in enumerate(self.sn):
    #   # surf is a vector of non-sorted nodes - e.g. [0,2,3]
    #   dummy = []
    #   for idx, i in enumerate(surf):
    #     self.sn[surf_idx][idx] = order[i]
    #     dummy += [self.nodes[order[i]]]
    #   self.surfaceNodeTags += [dummy]

    for surf_idx, surf in enumerate(self.sn):
      dummy = []
      for idx, i in enumerate(surf):
        dummy += [self.nodes[i]]
      self.surfaceNodeTags += [dummy]

  def get_node_info(self):
    self.entity_dims = []
    self.entity_tags = []
    for node in self.nodes:
      coords, parcoords, e_dim, e_tag = gmsh.model.mesh.getNode(node)
      try:
        self.node_coords = np.vstack([self.node_coords, coords])
      except:
        self.node_coords = coords
      self.entity_dims += [e_dim]
      self.entity_tags += [e_tag]
      self.entity_tag_pairs += [(e_dim, e_tag)]

class surface():
  def __init__(self, dim, etag, name, ptag, id: int):
    self.id = id # surface id used to refer to surfce internally
    self.dim = dim
    self.etag = etag
    self.ptag=ptag
    self.physicalName = name
    self.elements = []
    self.nodeTags, self.coords = gmsh.model.mesh.getNodesForPhysicalGroup(self.dim, self.ptag)
  def assign_element(self, element):
    self.elements += [element]

class volume(surface):
  def __init__(self, dim, etag, name, ptag, id):
    super().__init__(dim, etag, name, ptag, id)



surfaces = []
sid = 0
volumes = []
vid = 0
listed = []
for e in entities:
  dim = e[0]
  tag = e[1]
  # Get the mesh nodes for the entity (dim, tag):
  nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(dim, tag)

  # Get the upward and downward adjacencies of the entity.
  up, down = gmsh.model.getAdjacencies(dim, tag)

  # Get physical tags
  physicalTags = gmsh.model.getPhysicalGroupsForEntity(dim, tag)

  # determine if this is a labelled surface or possibly a boundary
  if dim == 2: # surface
    if len(physicalTags) >= 2:
      raise Exception("Surface cannot belong to more than 1 physical group.")
    elif len(physicalTags == 1):
      # is a surface with a name - e.g boundary
      name = gmsh.model.getPhysicalName(dim, physicalTags[0])
      this_surf = surface(dim=dim, etag=tag, name=name, ptag=physicalTags[0], id=sid)
      sid += 1
      surfaces += [this_surf]

  # determine if this is a volume
  if dim == 3:
    if len(physicalTags) >= 2:
      raise Exception("Volume cannot belong to more than 1 physical group.")
    elif len(physicalTags == 1):
      # is a volume with a name
      name = gmsh.model.getPhysicalName(dim, physicalTags[0])
      this_volume = volume(dim=dim, etag=tag, name=name, ptag=physicalTags[0], id=vid)
      vid += 1
      volumes += [this_volume]



In [640]:
# getting all elements in the mesh and adding some basic data

elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(-1, -1) # gets all elements in the mesh.
# see pdf page 367 - https://gmsh.info/dev/doc/texinfo/gmsh.pdf
# element type 15 = a point (dont care)
# element type 1 = 2 node line (useful for boundaries?)
# element type 3 = 4 node quadrangle (2d aka garbage)
# element type 4 = 4 node tetrahedon (3d keep)
# element type 7 = 5 node pyramid (3d keep)
elements = []
element_id = 0
for idx, t in enumerate(elementTypes):
  nt = nodeTags[idx] # node tags
  et = elementTags[idx] # element tags

  if t == 4: # 4 node tetrahedons
    numNodes = 4
    for nt_idx, this_node_tag in enumerate(nt):
      if nt_idx % numNodes == 0:
        # get node indexes for this element
        n0 = nt[nt_idx]
        n1 = nt[nt_idx+1]
        n2 = nt[nt_idx+2]
        n3 = nt[nt_idx+3]
        sn0 = [0, 2, 3] # nodes that make up each surface of this elements
        sn1 = [0, 1, 2]
        sn2 = [1, 2, 3]
        sn3 = [0, 1, 3]
        sn = [sn0, sn1, sn2, sn3]
        elements += [element(nodes=[n0, n1, n2, n3], ele_type=4, desc='4n_tet', eid=element_id, sn=sn)]
        element_id += 1
  if t == 7: # 5 node pyramid
    numNodes = 5
    for nt_idx, this_node_tag in enumerate(nt):
      if nt_idx % numNodes == 0:
        # get node indexes for this element
        n0 = nt[nt_idx]
        n1 = nt[nt_idx+1]
        n2 = nt[nt_idx+2]
        n3 = nt[nt_idx+3]
        n4 = nt[nt_idx+4]
        sn0 = [0,1,2,3]
        sn1 = [1,2,4]
        sn2 = [2,3,4]
        sn3 = [0,3,4]
        sn4 = [0,1,4]
        sn = [sn0, sn1, sn2, sn3, sn4]
        elements += [element(nodes=[n0, n1, n2, n3, n4], ele_type=7, desc='5n_pyr', eid=element_id, sn=sn)]
        element_id += 1


# Determining if each of the elements has a face on the boundary using elements and my boundary surfaces
for e in elements:
  e.determine_if_boundary(surfaces)





In [641]:
# getting the neighbors for all the elements in the mesh
internal_face_map = {}
for idx, element in enumerate(elements):
  for idx_this_elements_surf, surface in enumerate(element.surfaceNodeTags):
    if (element.is_boundary[idx_this_elements_surf] == False):
      surface_key = tuple(sorted(surface))
      if (surface_key not in internal_face_map):
        internal_face_map[surface_key] = []
      internal_face_map[surface_key].append(idx)

# # internal_face_map has tuples that are the keys to the dict where tuples are the nodeTags that make up the surface.
# internal_face_map values are [parent, neighbor] for the face.

# now assign neighbors for each surface in element
faces = []
face_id = 0
for nodeTags in internal_face_map.keys():
  owner_neighbor = internal_face_map[nodeTags]
  owner_id = owner_neighbor[0]
  neighbor_id = owner_neighbor[1]
  faces += [face(nodes=sorted(nodeTags), owner=owner_id, neighbor=neighbor_id, face_id=face_id)]

  elements[owner_id].assign_neighbor_info(nodeTags=nodeTags, neighbor_id=neighbor_id, face_id=face_id)
  elements[neighbor_id].assign_neighbor_info(nodeTags=nodeTags, neighbor_id=owner_id, face_id=face_id)
  face_id += 1



In [642]:
# now make faces for the face-boundaries - neighbor id are the negative of the boundary(surface) ids
for e in elements:
  for idx, b in enumerate(e.is_boundary):
    if b: # if True
      bnd_id = int(e.boundary_ids[idx])
      faces += [face(nodes=sorted(nodeTags), owner=e.eid, neighbor=-1, face_id = face_id)]
      faces[face_id].set_bndry(bnd_id)
      e.face_ids[idx] = face_id
      face_id += 1


In [644]:
# verify elements that all faces are either a boundary or a face
for e in  elements:
  e.verify_neighbors_and_faces()

In [645]:
# now we must calculate face centers for every face.
# faces includes internal and external faces.

# then face areas
# then do volume centroids based on book
# while doing volume centroids also do volumes.append
# getting close!